In [1]:
% matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import datetime

import sys
sys.path.append('C:\\project\\code')
import setting

import prepare_data0301

In [2]:
data = prepare_data0301._load_raw_data('../input/stock300.csv')

daily_return, weekly_return, t = prepare_data0301.main_total_vol(data)

Start reading CSV...
Done reading CSV...
Total-Vol: Formatting data...


In [3]:
ff = pd.read_csv('../input/famafrench.csv')
ff['date'] = pd.to_datetime(ff['date'], format='%Y%m%d')
ff.set_index('date', inplace=True)
ff.columns = ['MKT_RF', 'SMB', 'HML', 'RF']
weekly_ff = (1.0+ff).resample('W').prod() - 1.0
weekly_ff = weekly_ff.to_period('W')
weekly_data = pd.concat(
    [weekly_return, weekly_ff[['MKT_RF', 'SMB', 'HML', 'RF']]],
    axis=1,
    join='inner'
)

In [4]:

weekly_data['const'] = 1

weekly_data[weekly_return.columns] = (
    weekly_data[weekly_return.columns].sub(weekly_data['RF'], axis=0)
)


In [5]:
return_data = weekly_data
num_periods = 12

In [185]:
def _get_long_ff_vol_from_mthly(return_data, num_periods=12):
    global counter
    counter = 0
    
    permno_list = [
        x for x in return_data.columns
        if x not in ['const', 'RF', 'MKT_RF', 'HML', 'SMB']
        ]
    
    X = return_data[['const', 'MKT_RF', 'HML', 'SMB']] # copy
    ylist = return_data[permno_list]
    
    # convert to numpy array to speed up
    X_np = X.values
    ylist_np = ylist.values
    
    long_ff_vol = pd.DataFrame(index=return_data.index[num_periods-1:], columns=permno_list)
    
    for index1 in range(num_periods, len(return_data)+1):
        index0 = index1 - num_periods
        X_slice = X_np[index0:index1,:]
        ylist_slice = ylist_np[index0:index1,:]
        vol_one_period = np.apply_along_axis(prepare_data0301._ff_reg_long,
                                             0, ylist_slice, 
                                             ff_factors=X_slice, min_sample_size=10)
        long_ff_vol.loc[return_data.index[index1-1], :] = vol_one_period
    long_ff_vol.columns = long_ff_vol.columns.rename('PERMNO')
    return long_ff_vol.to_period('M')

In [44]:
temp = _get_long_ff_vol_from_mthly(weekly_data)

In [6]:
import time

In [8]:
start = time.time()
temp2 = prepare_data0301._get_long_ff_vol(weekly_data,);
end = time.time()
print(end-start)

321.6414670944214


In [12]:
def _get_long_ff_vol_from_weekly(return_data, num_periods=12):
    permno_list = [
        x for x in return_data.columns
        if x not in ['const', 'RF', 'MKT_RF', 'HML', 'SMB']
        ]

    X = return_data[['const', 'MKT_RF', 'HML', 'SMB']] # copy
    ylist = return_data[permno_list]

    # convert to numpy array to speed up
    X_np = X.values
    ylist_np = ylist.values
    
    # index for the X and Y value
    idx_weekly = return_data.index.to_series().astype(str)
    idx_weekly = idx_weekly.apply(lambda x: pd.to_datetime(x[-10:])).to_frame()
    idx_weekly['idx'] = np.arange(idx_weekly.shape[0])  # record the location in the original array
    idx_weekly = idx_weekly.set_index('date')
    
    # extract the index at the end of each month
    idx_mthly = idx_weekly.resample('M').last()
    
    # the container
    long_ff_vol = pd.DataFrame(index=idx_weekly.index[num_periods-1:], columns=permno_list)
    long_ff_vol = long_ff_vol.resample('M').last()
    
    # monthly index we loop through
    idx_use = idx_mthly[idx_mthly['idx'] > num_periods -1 ]
    idx_use_value = np.squeeze(idx_use.values)
    
    for i in range(idx_use.shape[0]):
        index1 = idx_use_value[i]
        index0 = index1 - num_periods
        X_slice = X_np[index0:index1,:]
        ylist_slice = ylist_np[index0:index1,:]
        vol_one_period = np.apply_along_axis(prepare_data0301._ff_reg_long,
                                             0, ylist_slice, 
                                             ff_factors=X_slice, min_sample_size=10)
        long_ff_vol.loc[idx_use.index[i], :] = vol_one_period
        
    long_ff_vol.columns = long_ff_vol.columns.rename('PERMNO')
    return long_ff_vol.to_period('M')

In [13]:
start = time.time()
temp = _get_long_ff_vol_from_weekly(weekly_data)
end = time.time()
print(end-start)

23.92636203765869


In [ ]:
# Debug the weekly total vol

In [2]:
data = prepare_data0301._load_raw_data('../input/stock300.csv')

Start reading CSV...
Done reading CSV...


In [3]:
daily_return = prepare_data0301._get_daily_return(data)

In [4]:
weekly_return = prepare_data0301._get_weekly_return(daily_return)

In [5]:
long_vol_24w = prepare_data0301._get_long_vol_from_weekly_return(weekly_return,24)

In [11]:
long_vol_24w.index = long_vol_24w.index.to_datetime()
long_vol_24w = long_vol_24w.resample('M').last().to_period('M')